# Experiment with CIFAR10
## Federated learning with vanila

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['AUTHGRAPH_VERBOSITY'] = '0'
import pickle
import time
import datetime
import random
import operator
import copy

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
tf.random.set_seed(6292)
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
import tensorflow_hub as hub
import numpy as np
import pandas as pd

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
today = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
today

In [ ]:
labelnames = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
labelnames

In [ ]:
dataset_root = os.path.abspath(os.path.expanduser('dataset-cifar10'))
dataset_root

In [ ]:
model_root = os.path.abspath(os.path.expanduser(f'model-cifar10-vanilla-{today}'))
model_root

In [ ]:
found = pd.read_csv('round-cifar10.csv', index_col='Clients')
found.fillna(-1, inplace=True)
found = found.astype('int')
found.drop(columns=['Total'], inplace=True)
found.drop(index=['Total'], inplace=True)
found

In [ ]:
found_round = np.unique(found.values).tolist()
found_round.remove(-1)
found_round.sort(reverse=False)
found_round

In [ ]:
found_label = dict()
for fr in found_round:
    label = found_label.get('all', dict())
    label[fr] = found.columns[((-1 < found.loc[:, :]) & 
                               (found.loc[:, :] <= fr)).any()].tolist()
    found_label['all'] = label
    for c in found.index:
        label = found_label.get(c, dict())
        label[fr] = found.columns[((-1 < found.loc[c, :]) & 
                                   (found.loc[c, :] <= fr))].tolist()
        found_label[c] = label
found_label

In [ ]:
found_client = dict()
for fr in found_round:
    found_client[fr] = found.index[((-1 < found.loc[:, :]) & 
                                    (found.loc[:, :] <= fr)).any(axis=1)].tolist()
found_client

In [ ]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)
target_size = (224, 224, 3)

In [ ]:
mobilenet_v2 = 'https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'
inception_v3 = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'
feature_extractor_model = mobilenet_v2

In [ ]:
feature_extractor_layer = hub.KerasLayer(feature_extractor_model, input_shape=target_size, trainable=True)
feature_extractor_layer

In [ ]:
model = tf.keras.Sequential([feature_extractor_layer,
                             tf.keras.layers.Dense(len(found_label['all'][0]))
                            ])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_path = os.path.join(model_root, f'{0}', 'global')
model.save(model_path)

In [ ]:
max_round = 120
cil_round = 5
epochs = 10
results = []
for fr in range(max_round+1):
    # Handle CIL
    cil = 0
    if fr in found_round:
        round_client = found_client[fr]
        round_label = found_label['all'][fr]
        ### model
        global_model_path = os.path.join(model_root, f'{fr}', 'global')
        model = tf.keras.models.load_model(global_model_path)
        feature_extractor_layer = hub.KerasLayer(feature_extractor_model, 
                                                 input_shape=target_size, 
                                                 trainable=True)
        new_model = tf.keras.Sequential([feature_extractor_layer,
                                         tf.keras.layers.Dense(len(round_label))
                                        ])
        new_model.compile(optimizer=tf.keras.optimizers.Adam(),
                          loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                          metrics=['accuracy'])
        weights = model.get_weights()
        diff = int(abs(weights[-1].shape[0]-len(found_label['all'][fr])))
        if diff != 0:
            weights[-2] = np.pad(weights[-2], (0, diff), mode='constant', constant_values=0)[:-diff,:]
            weights[-1] = np.pad(weights[-1], (0, diff), mode='constant', constant_values=0)
            for weight in weights[-2]:
                for idx in range(-diff, 0):
                    weight[idx] = random.uniform(-1, 1)
        new_model.set_weights(weights)
        feature_extractor_layer.trainable = False
        model_path = os.path.join(model_root, f'{fr}', 'global')
        new_model.save(model_path)
        cil = cil_round
        ### datasets
        datasets = dict()
        dataset_path = os.path.join(dataset_root, 'center', 'test')
        datasets['all'] = image_generator.flow_from_directory(dataset_path, classes=round_label,
                                                              target_size=target_size[:-1], shuffle=True,
                                                              follow_links=True)
        for client in round_client:
            train_dataset_path = os.path.join(dataset_root, 'scenario', f'{client}', f'{fr}', 'train')
            test_dataset_path = os.path.join(dataset_root, 'scenario', f'{client}', f'{fr}', 'test')
            datasets[client] = (image_generator.flow_from_directory(train_dataset_path, classes=round_label,
                                                                    target_size=target_size[:-1], shuffle=True,
                                                                    follow_links=True),
                                image_generator.flow_from_directory(test_dataset_path, classes=round_label,
                                                                    target_size=target_size[:-1], shuffle=True,
                                                                    follow_links=True))
    if cil > 0:
        cil = cil - 1
    global_model_path = os.path.join(model_root, f'{fr}', 'global')
    knowledges = []
    for client in round_client:
        model = tf.keras.models.load_model(global_model_path)
        if cil == 0:
            for layer in model.layers:
                layer.trainable = True
        model.fit(datasets[client][0], 
                  # validation_data=datasets[client][1],
                  epochs=epochs, verbose=0)
        model_path = os.path.join(model_root, f'{fr}', f'{client}')
        model.save(model_path)
        knowledges.append(model.get_weights())
    aggregates = copy.deepcopy(knowledges[0])
    for knowledge in knowledges[1:]:
        for i in range(len(knowledge)):
            aggregates[i] = aggregates[i] + knowledge[i]
    for i in range(len(aggregates)):
        aggregates[i] = aggregates[i] / len(knowledges)
    model = tf.keras.models.load_model(global_model_path)
    model.set_weights(aggregates)
    result = model.evaluate(datasets['all'], verbose=0)
    results.append(result)
    print(f'Federated round: {fr+1}, Result: {result}')
    global_model_path = os.path.join(model_root, f'{fr+1}', 'global')
    model.save(global_model_path)